In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np

import pandas as pd
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

import seaborn as sns
sns.set_context('notebook')
sns.set(style="whitegrid", font_scale=1.5)
sns.despine()

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.figsize'] = [20, 10]

from sklearn.linear_model import ElasticNetCV, TheilSenRegressor, RANSACRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error, max_error

import xgboost as xgb

import tensorflow as tf
tf.keras.backend.clear_session()

import tensorflowjs as tfjs

import onnxruntime as rt
from onnxconverter_common.data_types import FloatTensorType
from onnxmltools.convert import convert_sklearn, convert_keras

In [ ]:
dataset = pd.read_csv('measurements.csv', sep=';', decimal=',')
dataset = dataset.dropna()
dataset.drop(dataset.columns[[0, 1, 2]], axis = 1, inplace = True) 

max_tube_diameter = np.min(dataset[['AP cricoïde', 'AP fin de trachée', 'T fin de trachée']], axis=1)
dataset= dataset.iloc[:, :3]
dataset.drop(dataset.columns[[1]], axis = 1, inplace = True) 

dataset['max_tube_diameter'] = max_tube_diameter

cuffless_OD = np.array([2.9, 3.6, 4.2, 4.9, 5.5, 6.2, 6.8, 7.5, 8.2, 8.8, 9.6])
cuffed_OD = np.array([4.2, 5.5, 6.8, 7.5, 8.2, 8.8, 9.6, 10.2, 10.9, 11.5, 12.1, 12.8, 13.5])

In [ ]:
train_dataset = dataset.sample(frac=0.8, random_state=42)
test_dataset = dataset.drop(train_dataset.index)
train_dataset.describe().transpose()

In [ ]:
train_features = train_dataset.copy()
train_target = train_features.pop('max_tube_diameter')

test_features = test_dataset.copy()
test_target = test_features.pop('max_tube_diameter')

train_features.head()

In [ ]:
od = np.array([cuffless_OD[np.abs(cuffless_OD - mtd).argmin()] for mtd in test_dataset['max_tube_diameter']])
test_dataset['OD cuffless ref'] = cuffless_OD[np.maximum([j if i>0 else j-1 
                                for i, j in zip(np.sign(test_dataset['max_tube_diameter'] - od),
                                                [np.where(cuffless_OD == c)[0].item() for c in od])], 0)]


od = np.array([cuffed_OD[np.abs(cuffed_OD - mtd).argmin()] for mtd in test_dataset['max_tube_diameter']])
test_dataset['OD cuffed ref'] = cuffed_OD[np.maximum([j if i>0 else j-1 
                                for i, j in zip(np.sign(test_dataset['max_tube_diameter'] - od),
                                                [np.where(cuffed_OD == c)[0].item() for c in od])], 0)]

# Linear Regression

In [ ]:
reg=  TheilSenRegressor(random_state=42)
reg = make_pipeline(PolynomialFeatures(2), reg)
reg.fit(train_features, train_target)

In [ ]:
test_dataset['max_tube_diameter LR'] = reg.predict(test_features)

od = np.array([cuffless_OD[np.abs(cuffless_OD - mtd).argmin()] for mtd in test_dataset['max_tube_diameter LR']])
test_dataset['OD cuffless LR'] = cuffless_OD[np.maximum([j if i>0 else j-1 
                                for i, j in zip(np.sign(test_dataset['max_tube_diameter LR'] - od),
                                                [np.where(cuffless_OD == c)[0].item() for c in od])], 0)]


od = np.array([cuffed_OD[np.abs(cuffed_OD - mtd).argmin()] for mtd in test_dataset['max_tube_diameter LR']])
test_dataset['OD cuffed LR'] = cuffed_OD[np.maximum([j if i>0 else j-1 
                                for i, j in zip(np.sign(test_dataset['max_tube_diameter LR'] - od),
                                                [np.where(cuffed_OD == c)[0].item() for c in od])], 0)]

# Random Forest

In [ ]:
forest = RandomForestRegressor(random_state=42)

hyper_parameters = dict(n_estimators = [100, 300, 500, 800, 1200],
                        max_depth = [5, 8, 15, 25, 30, 50, 100, 150],
                        min_samples_split = [2, 5, 10, 15, 30, 50, 100],
                        min_samples_leaf = [1, 2, 5, 10, 15])

grid_search = GridSearchCV(forest, hyper_parameters, cv = 2, verbose = 1, n_jobs = 20)

best_rf = grid_search.fit(train_features, train_target)

print(grid_search.best_estimator_)

In [ ]:
test_dataset['max_tube_diameter RF'] = best_rf.predict(test_features)

od = np.array([cuffless_OD[np.abs(cuffless_OD - mtd).argmin()] for mtd in test_dataset['max_tube_diameter RF']])
test_dataset['OD cuffless RF'] = cuffless_OD[np.maximum([j if i>0 else j-1 
                                for i, j in zip(np.sign(test_dataset['max_tube_diameter RF'] - od),
                                                [np.where(cuffless_OD == c)[0].item() for c in od])], 0)]


od = np.array([cuffed_OD[np.abs(cuffed_OD - mtd).argmin()] for mtd in test_dataset['max_tube_diameter RF']])
test_dataset['OD cuffed RF'] = cuffed_OD[np.maximum([j if i>0 else j-1 
                                for i, j in zip(np.sign(test_dataset['max_tube_diameter RF'] - od),
                                                [np.where(cuffed_OD == c)[0].item() for c in od])], 0)]

## Gradient Boosting

In [ ]:
boosting = xgb.XGBRegressor(verbosity=2, nthread=20,
                            eta=0.2, gamma=0, max_depth=50, reg_lambda=1, reg_alpha=1, tree_method='exact',
                            objective='reg:squarederror', eval_metric='mae')

boosting.fit(train_features, train_target)

In [ ]:
test_dataset['max_tube_diameter GB'] = boosting.predict(test_features)

od = np.array([cuffless_OD[np.abs(cuffless_OD - mtd).argmin()] for mtd in test_dataset['max_tube_diameter GB']])
test_dataset['OD cuffless GB'] = cuffless_OD[np.maximum([j if i>0 else j-1 
                                for i, j in zip(np.sign(test_dataset['max_tube_diameter GB'] - od),
                                                [np.where(cuffless_OD == c)[0].item() for c in od])], 0)]


od = np.array([cuffed_OD[np.abs(cuffed_OD - mtd).argmin()] for mtd in test_dataset['max_tube_diameter GB']])
test_dataset['OD cuffed GB'] = cuffed_OD[np.maximum([j if i>0 else j-1 
                                for i, j in zip(np.sign(test_dataset['max_tube_diameter GB'] - od),
                                                [np.where(cuffed_OD == c)[0].item() for c in od])], 0)]

# Neural Network

In [ ]:
normalizer = tf.keras.layers.experimental.preprocessing.Normalization()
normalizer.adapt(np.array(train_features))

model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(2,), dtype=tf.float64),
    normalizer,
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=1)
])


model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.001),
    loss='mse',
    metrics=[tf.keras.metrics.RootMeanSquaredError(),
            tf.keras.metrics.MeanAbsoluteError()])

history = model.fit(x=train_features, y=train_target,
                    validation_data=(test_features, test_target),
                    epochs=300)

In [ ]:
test_dataset['max_tube_diameter NN'] = [model(np.array([row[1][:2]])).numpy().item() for row in test_dataset.iterrows()]

od = np.array([cuffless_OD[np.abs(cuffless_OD - mtd).argmin()] for mtd in test_dataset['max_tube_diameter NN']])
test_dataset['OD cuffless NN'] = cuffless_OD[np.maximum([j if i>0 else j-1 
                                for i, j in zip(np.sign(test_dataset['max_tube_diameter NN'] - od),
                                                [np.where(cuffless_OD == c)[0].item() for c in od])], 0)]


od = np.array([cuffed_OD[np.abs(cuffed_OD - mtd).argmin()] for mtd in test_dataset['max_tube_diameter NN']])
test_dataset['OD cuffed NN'] = cuffed_OD[np.maximum([j if i>0 else j-1 
                                for i, j in zip(np.sign(test_dataset['max_tube_diameter NN'] - od),
                                                [np.where(cuffed_OD == c)[0].item() for c in od])], 0)]

# Results

In [ ]:
pred = test_dataset[['age en mois', 'max_tube_diameter LR', "max_tube_diameter RF",
                     "max_tube_diameter GB", "max_tube_diameter NN"]]
pred = pred.set_index('age en mois', drop=True)
sns.lineplot(data=pred,  palette = "Dark2", linewidth=3);
sns.scatterplot(data=test_dataset, x='age en mois', y='max_tube_diameter');
sns.scatterplot(data=train_dataset, x='age en mois', y='max_tube_diameter', alpha=0.4);

In [ ]:
print('--- Cuffless predictions ---')
print()
print('LR Adapted diameter: {:.2%}'.format(sum(test_dataset['OD cuffless LR'] <= test_dataset['OD cuffless ref']) / len(test_dataset)))
print('RF Adapted diameter: {:.2%}'.format(sum(test_dataset['OD cuffless RF'] <= test_dataset['OD cuffless ref']) / len(test_dataset)))
print('GB Adapted diameter: {:.2%}'.format(sum(test_dataset['OD cuffless GB'] <= test_dataset['OD cuffless ref']) / len(test_dataset)))
print('NN Adapted diameter: {:.2%}'.format(sum(test_dataset['OD cuffless NN'] <= test_dataset['OD cuffless ref']) / len(test_dataset)))
print()
print('LR Exact diameter: {:.2%}'.format(sum(test_dataset['OD cuffless LR'] == test_dataset['OD cuffless ref']) / len(test_dataset)))
print('RF Exact diameter: {:.2%}'.format(sum(test_dataset['OD cuffless RF'] == test_dataset['OD cuffless ref']) / len(test_dataset)))
print('GB Exact diameter: {:.2%}'.format(sum(test_dataset['OD cuffless GB'] == test_dataset['OD cuffless ref']) / len(test_dataset)))
print('NN Exact diameter: {:.2%}'.format(sum(test_dataset['OD cuffless NN'] == test_dataset['OD cuffless ref']) / len(test_dataset)))
print()
print('--- Cuffed prediction ---')
print()
print('LR Adapted diameter: {:.2%}'.format(sum(test_dataset['OD cuffed LR'] <= test_dataset['OD cuffed ref']) / len(test_dataset)))
print('RF Adapted diameter: {:.2%}'.format(sum(test_dataset['OD cuffed RF'] <= test_dataset['OD cuffed ref']) / len(test_dataset)))
print('GB Adapted diameter: {:.2%}'.format(sum(test_dataset['OD cuffed GB'] <= test_dataset['OD cuffed ref']) / len(test_dataset)))
print('NN Adapted diameter: {:.2%}'.format(sum(test_dataset['OD cuffed NN'] <= test_dataset['OD cuffed ref']) / len(test_dataset)))
print()
print('LR Exact diameter: {:.2%}'.format(sum(test_dataset['OD cuffed LR'] == test_dataset['OD cuffed ref']) / len(test_dataset)))
print('RF Exact diameter: {:.2%}'.format(sum(test_dataset['OD cuffed RF'] == test_dataset['OD cuffed ref']) / len(test_dataset)))
print('GB Exact diameter: {:.2%}'.format(sum(test_dataset['OD cuffed GB'] == test_dataset['OD cuffed ref']) / len(test_dataset)))
print('NN Exact diameter: {:.2%}'.format(sum(test_dataset['OD cuffed NN'] == test_dataset['OD cuffed ref']) / len(test_dataset)))
print()
print('--- Prediction errors ---')
print('Mean Absolute Error')
print('LR: {}'.format(mean_absolute_error(test_dataset['max_tube_diameter'], test_dataset['max_tube_diameter LR'])))
print('RF: {}'.format(mean_absolute_error(test_dataset['max_tube_diameter'], test_dataset['max_tube_diameter RF'])))
print('GB: {}'.format(mean_absolute_error(test_dataset['max_tube_diameter'], test_dataset['max_tube_diameter GB'])))
print('NN: {}'.format(mean_absolute_error(test_dataset['max_tube_diameter'], test_dataset['max_tube_diameter NN'])))
print('Maximum Residual Error')
print('LR: {}'.format(max_error(test_dataset['max_tube_diameter'], test_dataset['max_tube_diameter LR'])))
print('RF: {}'.format(max_error(test_dataset['max_tube_diameter'], test_dataset['max_tube_diameter RF'])))
print('GB: {}'.format(max_error(test_dataset['max_tube_diameter'], test_dataset['max_tube_diameter GB'])))
print('NN: {}'.format(max_error(test_dataset['max_tube_diameter'], test_dataset['max_tube_diameter NN'])))

In [ ]:
def plot_loss(history, name):
    plt.plot(history.history['loss'], label='loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss (' + name + ')')
    plt.legend()
    plt.grid(True)
    
def plot_rmse(history, name):
    plt.plot(history.history['root_mean_squared_error'], label='rmse')
    plt.plot(history.history['val_root_mean_squared_error'], label='val_rmse')
    plt.xlabel('Epoch')
    plt.ylabel('Metric (' + name + ')')
    plt.legend()
    plt.grid(True)
    
print('MAE_val: {}'.format(min(history.history['val_mean_absolute_error'])))
print('RMSE_val: {}'.format(min(history.history['val_root_mean_squared_error'])))

plot_loss(history, 'MAE');
plt.show();
plot_rmse(history, 'RMSE');

In [ ]:
test_dataset.head()

In [ ]:
initial_type = [('float_input', FloatTensorType([None, train_features.shape[1]]))]

onx = convert_sklearn(reg, initial_types=initial_type)
with open("onnx/theil_sen.onnx", "wb") as f:
    f.write(onx.SerializeToString())

    
onx = convert_sklearn(best_rf, initial_types=initial_type)
with open("onnx/random_forest.onnx", "wb") as f:
    f.write(onx.SerializeToString())

# sess = rt.InferenceSession("onnx/random_forest.onnx")
# input_name = sess.get_inputs()[0].name
# label_name = sess.get_outputs()[0].name
# pred_onx = sess.run([label_name], {input_name: np.array(test_features).astype(np.float32)})[0]
# print(pred_onx)

tfjs.converters.save_keras_model(model, 'onnx/neural_network')